In [32]:
import requests
import xmltodict
import json
import datetime
import pandas as pd

In [33]:
# 보건복지부_코로나19 감염_현황
def get_Covid19_KR(end_day, start_day): 
    url='http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson' 
    api_key = '41cYgMWSvkL+jGmAZC/qLP0wimM1uABR3s2ehXnFqosoJBesTnXIFqj5UAlmPIRzClMLpQT8G+/UxwrF8v9HWw=='
    #ServiceKey는 url decode 한 값임. 
    params = { 
        'serviceKey' : api_key, 
        'pageNo' : '1',
        'numOfRows' : 10,
        'startCreateDt' : start_day, 
        'endCreateDt' : end_day, 
    } 
    res = requests.get(url, params=params) 
    if (res.status_code == 200): 
        # Ordered dictionary type 
        result = xmltodict.parse(res.text) 
        
        #dictionlay type 
        data = json.loads(json.dumps(result)) 
        leng = int(data['response']['body']['totalCount'])
        values, yesterdays = [], [0, 0, 0, 0]
        today_columns = ['Confirmed', 'Deaths', 'Recovered', 'Testeing']
        
        for i in reversed(range(leng)):
            day = data['response']['body']['items']['item'][i]["stateDt"]
            times = data['response']['body']['items']['item'][i]["stateTime"]
            t = datetime.datetime(int(day[0:4]), int(day[4:6]), int(day[6:]), int(times[0:2]),0,0,0)
            time = t.strftime("%Y%m%dT%H:%M:%SZ")
            Confirmed = int(data['response']['body']['items']['item'][i]['decideCnt'])
            Deaths = int(data['response']['body']['items']['item'][i]['deathCnt']) 
            Recovered = int(data['response']['body']['items']['item'][i]['clearCnt'])
            Testing = int(data['response']['body']['items']['item'][i]['examCnt'])   
            try:
                Tested = int(data['response']['body']['items']['item'][i]['accExamCnt'])
            except KeyError as e:
                Tested = 0
            try:
                Negative = int(data['response']['body']['items']['item'][i]['resutlNegCnt'])
            except KeyError as e:
                Negative = 0


            # 전날과 비교 값
            tConfirmed = Confirmed - yesterdays[0]
            tDeaths = Deaths - yesterdays[1]
            tRecovered = Recovered - yesterdays[2]
            tTesting = Testing - yesterdays[3]

            # 전날 데이터 최신화
            yesterdays[0] = Confirmed
            yesterdays[1] = Deaths
            yesterdays[2] = Recovered
            yesterdays[3] = Testing


            if len(values) > 1:
                if day != values[-1][0][0:8]:
                    values.append((time, Confirmed, Deaths, Recovered, Tested, Testing, Negative, tConfirmed, tDeaths, tRecovered, tTesting))
            else:
                values.append((time, Confirmed, Deaths, Recovered, Tested, Testing, Negative, tConfirmed, tDeaths, tRecovered, tTesting))


        columns = ['time', 'Confirmed', 'Deaths', 'Recovered', 'Tested', 'Testing', 'Negative', 'tConfirmed', 'tDeaths', 'tRecovered', 'tTesting']        
        df = pd.DataFrame(values, columns=columns)

    else: 
        print('res.status_code is NOT ok') 
    
    return df

In [35]:
today = datetime.datetime.now() 
day1 = today.strftime("%Y%m%d") 
# day2 = '20200203'
day2 = '20210224'

ret = get_Covid19_KR(day1, day2) # '20200203' 부터
ret.to_csv('data/Covid19_KR3.csv', sep=',', na_rep='NaN')
ret.head()

,time,Confirmed,Deaths,Recovered,Tested,Testing,Negative,tConfirmed,tDeaths,tRecovered,tTesting
0,20210224T00:00:00Z,88120,1576,79050,6510988,74988,6347880,88120,1576,79050,74988
1,20210225T00:00:00Z,88515,1581,79487,6551213,68672,6394026,395,5,437,-6316
2,20210226T00:00:00Z,88905,1585,79880,6590049,68942,6432202,390,4,393,270
3,20210227T00:00:00Z,89320,1595,80333,6627214,68989,6468905,415,10,453,47
4,20210228T00:00:00Z,89676,1603,80697,6649006,72889,6486441,356,8,364,3900


In [23]:
# 보건복지부_코로나19 시·도발생_현황
def get_Covid19_KR_Sido(end_day, start_day): 
    url='http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson' 
    api_key = '41cYgMWSvkL+jGmAZC/qLP0wimM1uABR3s2ehXnFqosoJBesTnXIFqj5UAlmPIRzClMLpQT8G+/UxwrF8v9HWw=='
    #ServiceKey는 url decode 한 값임. 
    params = { 
        'serviceKey' : api_key, 
        'pageNo' : '1',
        'numOfRows' : 10,
        'startCreateDt' : start_day, 
        'endCreateDt' : end_day, 
    } 
    res = requests.get(url, params=params) 
    if (res.status_code == 200): 
        # Ordered dictionary type 
        result = xmltodict.parse(res.text) 
        #dictionlay type 
        data = json.loads(json.dumps(result))         
        leng = int(data['response']['body']['totalCount'])
        values = []
        # print('지역\t확진자\t사망자\t격리해제\t발생률*')
        for i in range(leng):
            day = data['response']['body']['items']['item'][i]["stdDay"]
            times = day.split()
            t = datetime.datetime(int(times[0][0:-1]), int(times[1][0:-1]), int(times[2][0:-1]), int(times[3][0:-1]),0,0,0)
            time = t.strftime("%Y%m%dT%H:%M:%SZ")
            City = data['response']['body']['items']['item'][i]['gubun']
            Confirmed = data['response']['body']['items']['item'][i]['defCnt']
            Death = data['response']['body']['items']['item'][i]['deathCnt']
            Incidence = data['response']['body']['items']['item'][i]['qurRate']
            try:
                Recovered = data['response']['body']['items']['item'][i]['isolClearCnt']
            except KeyError as e:
                Recovered = 0
                
            values.append((time, City, Confirmed, Death, Recovered, Incidence))

        values = list(reversed(values))
        columns = ['time', 'City', 'Confirmed', 'Death', 'Recovered', 'Incidence']
        df = pd.DataFrame(values, columns=columns)
        
    else: 
        print('res.status_code is NOT ok') 

    return df

In [24]:
today = datetime.datetime.now() 
day1 = today.strftime("%Y%m%d") 
# day2 = '20200304'
day2 = '20210225'


ret = get_Covid19_KR_Sido(day1, day2) # '20200304' 부터
ret.to_csv('data/Covid19_KR_Sido2.csv', sep=',', na_rep='NaN')
ret.head()

,time,City,Confirmed,Death,Recovered,Incidence
0,20210225T00:00:00Z,합계,88516,1581,79487,170.72
1,20210225T00:00:00Z,서울,27857,373,24652,286.20
2,20210225T00:00:00Z,부산,3198,105,2814,93.73
3,20210225T00:00:00Z,대구,8602,213,8264,353.05
4,20210225T00:00:00Z,인천,4376,52,4048,148.03


In [25]:
# 보건복지부_코로나19해외발생_현황
def get_Covid19_GB(end_day, start_day): 
    url='http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19NatInfStateJson' 
    api_key = '41cYgMWSvkL+jGmAZC/qLP0wimM1uABR3s2ehXnFqosoJBesTnXIFqj5UAlmPIRzClMLpQT8G+/UxwrF8v9HWw=='
    #ServiceKey는 url decode 한 값임. 
    params = { 
        'serviceKey' : api_key, 
        'pageNo' : '1',
        'numOfRows' : 10,
        'startCreateDt' : start_day, 
        'endCreateDt' : end_day, 
    } 
    res = requests.get(url, params=params) 
    if (res.status_code == 200): 
        # Ordered dictionary type 
        result = xmltodict.parse(res.text) 
        #dictionlay type 
        data = json.loads(json.dumps(result))         
        leng = int(data['response']['body']['totalCount'])
        values = []
        print('나라\t확진자\t사망자\t사망률')
        for i in range(leng):
            day = data['response']['body']['items']['item'][i]["stdDay"]
            times = day.split()
            t = datetime.datetime(int(times[0][0:-1]), int(times[1][0:-1]), int(times[2][0:-1]), int(times[3][0:-1]),0,0,0)
            time = t.strftime("%Y%m%dT%H:%M:%SZ")
            nationNm = data['response']['body']['items']['item'][i]['nationNm']
            natDefCnt = data['response']['body']['items']['item'][i]['natDefCnt']
            natDeathCnt = data['response']['body']['items']['item'][i]['natDeathCnt']

            if data['response']['body']['items']['item'][i]['natDeathRate'] == 'NaN':
                natDeathRate = 0.0
            else:
                natDeathRate = data['response']['body']['items']['item'][i]['natDeathRate']
            
            if data['response']['body']['items']['item'][i]['nationNmEn'] == None:
                nationNmEn = 'missing'
            else:
                nationNmEn = data['response']['body']['items']['item'][i]['nationNmEn']
            
            
            values.append((time, nationNm, nationNmEn, natDefCnt, natDeathCnt, natDeathRate))

        values = list(reversed(values))
        values = values[1:]
        columns = ['time', 'nationNm', 'nationNmEn', 'natDefCnt', 'natDeathCnt', 'natDeathRate']
        df = pd.DataFrame(values, columns=columns)
        
    else: 
        print('res.status_code is NOT ok') 

    return df

In [26]:
today = datetime.datetime.now() 
day1 = today.strftime("%Y%m%d") 
day2 = '20200316'
day2 = '20210225'

ret = get_Covid19_GB(day1, day2) # '20200316' 부터
ret.to_csv('data/Covid19_GB2.csv', sep=',', na_rep='NaN')
ret.head()

나라	확진자	사망자	사망률


,time,nationNm,nationNmEn,natDefCnt,natDeathCnt,natDeathRate
0,20210225T09:00:00Z,중국,China,89871,4636,5.1585049682
1,20210225T09:00:00Z,홍콩,Hong Kong,10913,198,1.8143498579
2,20210225T09:00:00Z,대만,Taiwan,946,9,0.9513742071
3,20210225T09:00:00Z,마카오,Macau,48,0,0.0
4,20210225T09:00:00Z,일본,Japan,427467,7584,1.7741720413


In [27]:
# 보건복지부_코로나19 연령별·성별감염_현황
def get_Covid19_KR_GenAge(end_day, start_day): 
    url='http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson' 
    api_key = '41cYgMWSvkL+jGmAZC/qLP0wimM1uABR3s2ehXnFqosoJBesTnXIFqj5UAlmPIRzClMLpQT8G+/UxwrF8v9HWw=='
    #ServiceKey는 url decode 한 값임. 
    params = { 
        'serviceKey' : api_key, 
        'pageNo' : '1',
        'numOfRows' : 10,
        'startCreateDt' : start_day, 
        'endCreateDt' : end_day, 
    } 
    res = requests.get(url, params=params) 
    if (res.status_code == 200): 
        # Ordered dictionary type 
        result = xmltodict.parse(res.text) 
        #dictionlay type 
        data = json.loads(json.dumps(result))         
        leng = int(data['response']['body']['totalCount'])
        values = []
        for i in range(leng):
            t = data['response']['body']['items']['item'][i]['createDt'].split()
            day = t[0].split('-')
            times =  t[1].split(':')
            t = datetime.datetime(int(day[0]), int(day[1]), int(day[2]), int(times[0]),0,0,0)
            time = t.strftime("%Y%m%dT%H:%M:%SZ")
            gubun = data['response']['body']['items']['item'][i]['gubun']               # 구분
            confCase = data['response']['body']['items']['item'][i]['confCase']         # 확진자
            confCaseRate = data['response']['body']['items']['item'][i]['confCaseRate'] # 확진률
            death = data['response']['body']['items']['item'][i]['death']               # 사망자
            deathRate = data['response']['body']['items']['item'][i]['deathRate']       # 사망률
            criticalRate = data['response']['body']['items']['item'][i]['criticalRate'] # 치명률

            values.append((time, gubun, confCase, confCaseRate, death, deathRate, criticalRate))

        values = list(reversed(values))
        columns = ['time', 'gubun', 'confCase', 'confCaseRate', 'death', 'deathRate', 'criticalRate']
        df = pd.DataFrame(values, columns=columns)
        
    else: 
        print('res.status_code is NOT ok') 

    return df

In [28]:
today = datetime.datetime.now() 
day1 = today.strftime("%Y%m%d") 
day2 = '20200408'
day2 = '20210225'

ret = get_Covid19_KR_GenAge(day1, day2) # '20200408' 부터
ret.to_csv('data/Covid19_KR_GenAge2.csv', sep=',', na_rep='NaN')
ret

,time,gubun,confCase,confCaseRate,death,deathRate,criticalRate
0,20210225T14:00:00Z,남성,43633,49.29,786,49.72,1.8
1,20210225T14:00:00Z,여성,44883,50.71,795,50.28,1.77
2,20210225T14:00:00Z,80 이상,4340,4.9,895,56.61,20.62
3,20210225T14:00:00Z,70-79,6740,7.61,432,27.32,6.41
4,20210225T14:00:00Z,60-69,13862,15.66,183,11.57,1.32
5,20210225T14:00:00Z,50-59,16471,18.61,52,3.29,0.32
6,20210225T14:00:00Z,40-49,12683,14.33,12,0.76,0.09
7,20210225T14:00:00Z,30-39,11529,13.02,6,0.38,0.05
8,20210225T14:00:00Z,20-29,13372,15.11,1,0.06,0.01
9,20210225T14:00:00Z,10-19,5944,6.72,0,0.00,0
